In [78]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle


In [79]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [80]:
df=df.drop(['CustomerId','RowNumber','Surname'],axis=1)

In [81]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [82]:
## label encoding and one hot encoding the Gender and the Geography 
le_gender=LabelEncoder()
df['Gender']=le_gender.fit_transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


implementing  the one hot encoding for the geography cause we have 3 different
features for that which is not good for label encoding cause the neural network 
might take it as a  prioritization basis for the development of the model


In [83]:
#initialize the one hot encoder
ohe=OneHotEncoder(sparse_output=False)
## fit and transform
encoded=ohe.fit_transform(df[['Geography']])

## create dataframes with the column name 
encoded_df=pd.DataFrame(encoded,columns=ohe.get_feature_names_out(['Geography']))
encoded_df

## Drop the original column name and concatenate the new one 
df=df.drop(['Geography'],axis=1)
df=pd.concat([df,encoded_df],axis=1)

In [84]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [85]:
X=df.drop('EstimatedSalary',axis=1)
y=df['EstimatedSalary']

In [86]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,1,0.0,1.0,0.0


In [87]:
y

0       101348.88
1       112542.58
2       113931.57
3        93826.63
4        79084.10
          ...    
9995     96270.64
9996    101699.77
9997     42085.58
9998     92888.52
9999     38190.78
Name: EstimatedSalary, Length: 10000, dtype: float64

In [88]:
with open('le_gender.pkl','wb') as file:
    pickle.dump(le_gender,file)
with open("ohe_geo","wb") as file:
    pickle.dump(encoded_df,file)
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

In [89]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)


In [90]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [91]:
### ! training the ANN for the regression problem statement


In [94]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [97]:
model = Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)),
    Dense(32,activation="relu"),
    Dense(1) # default activation function is linear its spsecifically for regression

])

## compile the model
model.compile(optimizer="adam",loss="mean_absolute_error",metrics=['mae'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                832       
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [98]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

## setting up the tensorboard
log_dir="regressionLogs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
TensorBoard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [99]:
EarlyStopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [100]:
##Training the model
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[EarlyStopping_callback,TensorBoard_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 5ms/step - loss: 100373.9219 - mae: 100373.9219 - val_loss: 98499.4062 - val_mae: 98499.4062
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 99538.1250 - mae: 99538.1250 - val_loss: 96796.6875 - val_mae: 96796.6875
Epoch 3/100
250/250 [==============================] - 0s 998us/step - loss: 96561.2109 - mae: 96561.2109 - val_loss: 92425.0156 - val_mae: 92425.0156
Epoch 4/100
250/250 [==============================] - 0s 931us/step - loss: 90762.6719 - mae: 90762.6719 - val_loss: 85272.0391 - val_mae: 85272.0391
Epoch 5/100
250/250 [==============================] - 0s 926us/step - loss: 82418.2812 - mae: 82418.2812 - val_loss: 76166.3594 - val_mae: 76166.3594
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 72847.4375 - mae: 72847.4375 - val_loss: 66978.8594 - val_mae: 66978.8594
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 63965.3281 - mae: 63965

In [101]:
%load_ext tensorboard

In [104]:
%tensorboard --logdir regressionLogs/git

Reusing TensorBoard on port 6008 (pid 91589), started 0:00:45 ago. (Use '!kill 91589' to kill it.)

In [106]:
## evaluate the model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f"Test mae  : {test_mae}")

63/63 [==============================] - 0s 382us/step - loss: 50299.8633 - mae: 50299.8633
Test mae  : 50299.86328125


In [107]:
model.save("regression_model.h5")

/Users/shantanukumar/Documents/codes/ANN-Basic/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
